## Project Scope: Building a Spam Filter using Naive Bayes

In this notebook, we will be creating a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to build a model that can accurately classify SMS messages as spam or ham (not spam) with an accuracy of at least 90%.

The original dataset has already labeled messages as spam or ham, so we will not be using any labels in our model.

### Naive Bayes Explained

The Naive Bayes algorithm is a simple yet powerful technique used primarily for classification tasks (e.g., spam detection, sentiment analysis). It belongs to a family of probabilistic classifiers that are based on applying Bayes' Theorem with a strong, or "naive," independence assumption.

1. Conditional probability
Witten as $P(A|B)$ where an event $A$ occurs given that event $B$ has occurred. For example, $P(\text{Spam}|\text{"Free"})$ is the probability that an email is spam, given that the word "Free" appears in the email.
$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$
Where $P(A \cap B)$ is the probability that both $A$ and $B$ occur.

2. Bayes' Theorem
Bayes' Theorem allows us to flip the conditional probability $P(A|B)$ to $P(B|A)$. For example:
$$P(B|A) = \frac{P(A|B)P(B)}{P(A)}$$

In the context of classification, this can be restated as $$P(Class|Features) = \frac{P(Features|Class)P(Class)}{P(Features)}$$

Where:
* $P(Class|Features)$: Posterior Probability, what we want to find, the probability that the data belongs to a certain class (what is the probability that the data is spam given certain features).
* $P(Features|Class)$: Likelihood, the probability of observing the given features, given the class is true.
* $P(Class)$: Prior Probability, the initial probability of the class before any evidence is seen. 
* $P(Features)$: Evidence, the probability of observing the features, which acts as a scaling constant. 

3. Naive Assumption
Naive Bayes uses conditional probability in the form of Bayes' Theorem to find the probability of a hypothesis (our classification) given the evidence (our data features).

The reason the algorithm is called "Naive" is due to its simplifying assumption: it assumes that all features used for classification are independent of each other, given the class.

For example, when classifying an email, it assumes that the presence of the word "coupon" is completely independent of the presence of the word "money," given the email is spam.

Mathematically, this means the likelihood term $P(\text{Features}|\text{Class})$ can be decomposed into a product of individual conditional probabilities:$$P(x_1, x_2, \dots, x_n|\text{Class}) = P(x_1|\text{Class}) \times P(x_2|\text{Class}) \times \dots \times P(x_n|\text{Class})$$

### Take a peek at the data

In [3]:
import pandas as pd
sms = pd.read_csv("SMSSpamCollection", sep='\t', header=None, names=['Label', 'SMS'])
sms

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [9]:
#check how many ham, and spam messages we have
print(sms['Label'].value_counts(normalize=True))

Label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64


### Training and Testing Data

We will split the dataset into 80% training and 20% testing.

In [15]:
#shuffle the dataset
shuffled = sms.sample(frac=1, random_state=42)

#split the dataset into training and testing sets
train_size = int(len(shuffled) * 0.8)
train = shuffled[:train_size].reset_index(drop=True)
test = shuffled[train_size:].reset_index(drop=True)

print("Training set size:", len(train))
print("Testing set size:", len(test))

Training set size: 4457
Testing set size: 1115


we would expect that in the training set and test set, the percentage of spam is about the same as in the full dataset(13%). 

In [16]:
print(train['Label'].value_counts(normalize = True))

Label
ham     0.866951
spam    0.133049
Name: proportion, dtype: float64


In [11]:
print(test['Label'].value_counts(normalize=True))

Label
ham     0.861883
spam    0.138117
Name: proportion, dtype: float64


### Before training: Data cleaning

As exlpained before, we would want to know the occurance of features given a certain class. $P(\text{Features}|\text{Class})$ can be decomposed into a product of individual conditional probabilities:$$P(x_1, x_2, \dots, x_n|\text{Class}) = P(x_1|\text{Class}) \times P(x_2|\text{Class}) \times \dots \times P(x_n|\text{Class})$$

Here, within an email labeled "spam", there could be words like "SECRET", "PRIZE", "CLAIM", "SECRET", "PRIZE", and "NOW". Where you could view each word as independent event that occurred given the class "spam".

So, what we want to do is to tokenize each word in the email and count the number of times it occurred given the class "spam" or "ham". 

#### Step 1: Remove punctuation and convert to lowercase in training data

In [19]:
import string
punctuation = '[' +string.punctuation + ']'
train['SMS'] = train['SMS'].str.replace(punctuation, ' ', regex=True)
train['SMS'] = train['SMS'].str.lower()
train.head()


,Label,SMS
0,ham,squeeeeeze this is christmas hug if u lik ...
1,ham,and also i ve sorta blown him off a couple tim...
2,ham,mmm thats better now i got a roast down me i...
3,ham,mm have some kanji dont eat anything heavy ok
4,ham,so there s a ring that comes with the guys cos...


#### Step 2: Tokenizing the vocabulary

In [20]:
train['SMS'] = train['SMS'].str.split()
vocab = []
for i in train['SMS']:
    for j in i:
        vocab.append(j)
vocab = set(vocab) #reduce replicates
vocab = list(vocab)


In [21]:
len(vocab)

7890

#### Step 3: Create a new dataframe for training that counts the number of words in each message

In [23]:
train['SMS']

0       [squeeeeeze, this, is, christmas, hug, if, u, ...
1       [and, also, i, ve, sorta, blown, him, off, a, ...
2       [mmm, thats, better, now, i, got, a, roast, do...
3       [mm, have, some, kanji, dont, eat, anything, h...
4       [so, there, s, a, ring, that, comes, with, the...
                              ...                        
4452    [solve, d, case, a, man, was, found, murdered,...
4453    [maybe, if, you, woke, up, before, fucking, 3,...
4454    [me, hungry, buy, some, food, good, lei, but, ...
4455         [wow, v, v, impressed, have, funs, shopping]
4456    [yo, you, around, a, friend, of, mine, s, look...
Name: SMS, Length: 4457, dtype: object

In [24]:
vocab

['innu',
 '31p',
 'kadeem',
 'shorter',
 'endless',
 'case',
 'father',
 'terrible',
 'ag',
 'lately',
 'monos',
 '88088',
 'leaves',
 'peace',
 'izzit',
 'docks',
 '220cm2',
 'ummifying',
 'mrng',
 'minded',
 'ali',
 'adi',
 'convey',
 'ajith',
 'add',
 'bought',
 'coin',
 'staring',
 'linear',
 'ibiza',
 'explosive',
 '50pmmorefrommobile2bremoved',
 '2day',
 '09061702893',
 '69669',
 'offcampus',
 'lengths',
 'layin',
 'meaningless',
 'bitching',
 'recent',
 'bot',
 '08712402902',
 'disturbance',
 'law',
 'ahhhh',
 'samus',
 'photoshop',
 'leaving',
 'lighters',
 'sonyericsson',
 '08714712379',
 'representative',
 'necessity',
 'saturday',
 'sunoco',
 'lucy',
 'debating',
 'student',
 'like',
 'sweets',
 'dolls',
 'nothing',
 'dificult',
 'sugar',
 'yeh',
 'temales',
 'childish',
 '09064019014',
 'among',
 'recognises',
 'giving',
 'asia',
 'muht',
 '150',
 '09050000460',
 'nb',
 'pre',
 'breathe1',
 '7876150ppm',
 'infernal',
 'favor',
 'fringe',
 'diwali',
 'table',
 'pobox45w2tg15

In [28]:
#create a dictionary that holds word and their frequency
word_count_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocab}

#add up the number of times each word appears in each sms
for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_count_per_sms[word][index] += 1

word_counts = pd.DataFrame(word_count_per_sms)
word_counts.head()

,innu,31p,kadeem,shorter,endless,case,father,terrible,ag,lately,...,golddigger,listed,na,09050090044,unjalur,txt250,london,shampain,22,loyalty
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#combine original train and word_counts dataframes
train_cleaned = pd.concat([train, word_counts], axis=1)
train_cleaned.head()

,Label,SMS,innu,31p,kadeem,shorter,endless,case,father,terrible,...,golddigger,listed,na,09050090044,unjalur,txt250,london,shampain,22,loyalty
0,ham,"[squeeeeeze, this, is, christmas, hug, if, u, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[and, also, i, ve, sorta, blown, him, off, a, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[mmm, thats, better, now, i, got, a, roast, do...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[mm, have, some, kanji, dont, eat, anything, h...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[so, there, s, a, ring, that, comes, with, the...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Spam Filter Creation

Now we have a training set that contains the frequency of each word in the spam and ham messages, we also need 4 key probabilities to make predictions:
1. P(Spam)
2. P(Ham)
3. P(word | Spam)
4. P(word | Ham)

The 2 main questions we need to answer are:
1. What is the probability of a message being spam given the words in the message $P(Spam | word)$?
2. What is the probability of a message being ham given the words in the message $P(Ham | word)$?

Eventually, the filter that we build will predict the class (spam or ham) that maximizes this probability $P(Class|word)$.

#### Estimation formula - Pre calculation

In machine learning, we don't calculate probabilities from a theoretical space; we estimate them from our observed data (the training set). The probability of an event is estimated by its Relative Frequency: $$P(\text{Event}) \approx \frac{\text{Count of Event Occurrences}}{\text{Total Number of Trials}}$$


*Note: In order to prevent 0 probabilities, we will use the Laplace smoothing technique -> alpha = 1*

$$\mathbf{P(w_i|\text{Spam})} = \frac{N_{w_i|\text{Spam}} + \alpha}{N_{\text{Spam}} + \alpha \cdot N_{\text{Vocabulary}}}$$

$$\mathbf{P(w_i|\text{Ham})} = \frac{N_{w_i|\text{Ham}} + \alpha}{N_{\text{Ham}} + \alpha \cdot N_{\text{Vocabulary}}}$$

In [50]:
is_spam = train['Label'] == 'spam'
is_ham = train['Label'] == 'ham'
spam = train[is_spam]
ham = train[is_ham]
spam_indices = spam.index.tolist()
ham_indices = ham.index.tolist()


#P(Spam) & P(Ham)
p_spam = len(spam) / len(train)
p_ham = len(ham) / len(train)

#N_Spam & N_Ham -> Number of words in spam and ham
n_spam = spam['SMS'].apply(len).sum()
n_ham = ham['SMS'].apply(len).sum()

#N_vocabulary -> Number of unique words in the vocabulary
n_vocab = len(vocab) #vocab is already unique

alpha = 1

In [68]:
#we want to get the parameter P(word|spam) and P(word|ham)
word_counts_per_sms = {unique_word: [0] * len(train) for unique_word in vocab}

for word in vocab:
    # Get the list of counts for the current word across ALL messages
    all_counts = word_counts_per_sms[word]
    
    # Calculate N_wi|Spam: Sum the counts only at the indices that correspond to spam messages
    N_wi_spam = sum(all_counts[i] for i in spam_indices)
    
    # Calculate N_wi|Ham: Sum the counts only at the indices that correspond to ham messages
    N_wi_ham = sum(all_counts[i] for i in ham_indices)
    
    # Apply the formula using the corrected count
    parameter_spam[word] = (N_wi_spam + alpha) / (n_spam + alpha * n_vocab)
    parameter_ham[word] = (N_wi_ham + alpha) / (n_ham + alpha * n_vocab)

#### Putting it all together - Classifying a new message

From the calculation above, we have get 4 parameters:
1. P(Spam)
2. P(Ham)
3. P(word | Spam)
4. P(word | Ham)

Now we can use these parameters to classify new messages. The spam filter can be implemented as follows:
* Takes a message (w1, w2, w3, ...) as input
* Calculate P(Spam | w1, w2, w3, ...) and P(Ham | w1, w2, w3, ...)
* If P(Spam | w1, w2, w3, ...) > P(Ham | w1, w2, w3, ...), classify as Spam
* Otherwise, classify as Ham

We have derived from the above expression that there is a way to flip from P(word|Class) to P(Class|word):
$$P(\text{Class}|\text{word}) = \frac{P(\text{word} \cap \text{Class})}{P(\text{Class})} \times \frac{P(\text{Class})}{P(\text{word})}$$

Essentially, we can derive the P(word|Class) first, then do multiplication.



In [69]:
#P(Spam) = p_spam
#P(Ham) = p_ham
#P(word|Spam) = parameter_spam
#P(word|Ham) = parameter_ham
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]
            
        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')


<>:12: SyntaxWarning: invalid escape sequence '\W'
<>:12: SyntaxWarning: invalid escape sequence '\W'
C:\Users\USER\AppData\Local\Temp\ipykernel_25144\1213140241.py:12: SyntaxWarning: invalid escape sequence '\W'
  message = re.sub('\W', ' ', message)


The goal of the loop is to calculate the Posterior Probability for the Spam class using the following proportional relationship derived from Bayes' Theorem:$$P(\text{Spam}|\text{message}) \propto P(\text{Spam}) \times P(w_1|\text{Spam}) \times P(w_2|\text{Spam}) \times \dots \times P(w_n|\text{Spam})$$

In [70]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 7.433297973622768e-41
P(Ham|message): 4.2347591924561483e-44
Label: Spam


In [71]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 3.9267456236621384e-32
P(Ham|message): 1.7845643898206466e-34
Label: Spam


### Test it out

In [72]:
test['predicted'] = test['SMS'].apply(classify)
test.head()

P(Spam|message): 9.025232141425058e-28
P(Ham|message): 1.158467819295971e-29
Label: Spam
P(Spam|message): 7.433297973622768e-41
P(Ham|message): 4.2347591924561483e-44
Label: Spam
P(Spam|message): 7.433297973622768e-41
P(Ham|message): 4.2347591924561483e-44
Label: Spam
P(Spam|message): 3.9267456236621384e-32
P(Ham|message): 1.7845643898206466e-34
Label: Spam
P(Spam|message): 7.433297973622768e-41
P(Ham|message): 4.2347591924561483e-44
Label: Spam
P(Spam|message): 2.3201737486796026e-119
P(Ham|message): 1.0104089651085971e-130
Label: Spam
P(Spam|message): 9.545015512697406e-76
P(Ham|message): 1.3428104327448214e-82
Label: Spam
P(Spam|message): 2.074359355385135e-23
P(Ham|message): 7.520309695741724e-25
Label: Spam
P(Spam|message): 9.025232141425058e-28
P(Ham|message): 1.158467819295971e-29
Label: Spam
P(Spam|message): 1.7084692062574568e-36
P(Ham|message): 2.749036277374833e-39
Label: Spam
P(Spam|message): 6.474749349173315e-102
P(Ham|message): 1.7943377631735853e-111
Label: Spam
P(Spam|

,Label,SMS,predicted
0,ham,I fetch yun or u fetch?,None
1,ham,Was playng 9 doors game and gt racing on phone...,None
2,ham,I dont thnk its a wrong calling between us,None
3,ham,All e best 4 ur exam later.,None
4,ham,Hey what how about your project. Started aha da.,None


In [ ]:
#Accuracy 
correct = 0
for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
print(correct / len(test))

0.0
